In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()

TTC = TimeTaggerController()
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(4)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(3)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()




ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [26]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   -29 ps 	/    -8.7 mm
Bob_T   :  -178 ps 	/   -53.4 mm
Bob_R   :   -70 ps 	/   -21.0 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   -11 ps 	/    -3.3 mm
Bob_T   :     1 ps 	/     0.3 mm
Bob_R   :    -6 ps 	/    -1.8 mm


In [42]:
n_values = 150
trace_width = 3
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

In [44]:
KMC.rotate_simulataneously(0, 45, wait_for_completion=False)

In [45]:
plusAngles = [0, -22.5, -11.25, 11.25] # for triplett |HH> + |VV> initial state (plate looks backwards)
minusAngles = [0, -22.5, 11.25, -11.25] # for triplett |HH> + |VV> initial state (plate looks forwards/correct)
TTC.measureS(CHSH_angles=plusAngles, integration_time_per_basis_setting_SI=2, TTSimulator=None, coincidence_window_SI=0.5e-9, debug=True)


corr[a,b] = 0.81759
	N[|T,T>]=   977	(0.159)
	N[|T,R>]=   295	(0.048)
	N[|R,T>]=   264	(0.043)
	N[|R,R>]=  4593	(0.749)

corr[a,B] = 0.52055
	N[|T,T>]=   876	(0.146)
	N[|T,R>]=   785	(0.131)
	N[|R,T>]=   650	(0.109)
	N[|R,R>]=  3675	(0.614)

corr[A,b] = 0.59649
	N[|T,T>]=   778	(0.155)
	N[|T,R>]=   479	(0.095)
	N[|R,T>]=   533	(0.106)
	N[|R,R>]=  3226	(0.643)

corr[A,B] = -0.62987
	N[|T,T>]=   119	(0.025)
	N[|T,R>]=  2355	(0.486)
	N[|R,T>]=  1595	(0.329)
	N[|R,R>]=   778	(0.161)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.5645018358575293


In [6]:
my_ip = '192.168.0.2'
clemens_ip = "192.168.0.3"
enar_ip = "192.168.0.5"
my_port = 7401 
clemens_port = 7401
enar_port = 10000

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip=clemens_ip, port=clemens_port, name='Clemens')
Enar = OSCTarget(ip=enar_ip, port=enar_port, name="Enar")

# Start the server
OSC.start_server()


OSC Server Started. Serving on ('192.168.0.2', 7401)
----------------------------------------------------


In [8]:
# Global flag to control the loop
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = plusAngles
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, -11.25), (0, 11.25), (-22.5, -11.25), (-22.5, 11.25)], # set for plusAngles
                    # ab, AB
                   'Q_diagonal': [(0, 11.25), (-22.5, -11.25)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

# for this perf we only want Q_all
angle_pairs = angle_pair_dict['Q_all']

# setup system with correct initial angles
KMC.rotate_simulataneously(*angle_pairs[0])

def send_results(result, theta_a, theta_b):
    resultClemensFormat = [1 if theta_a == Q_angles[0] else 2, 1 if theta_b == Q_angles[2] else 2 ,*resultTranslate[result]]
    OSC.send_measurement(Clemens, resultClemensFormat)
    OSC.send_measurement(Enar, resultClemensFormat)

def performance_cycle_both(cycles):
    """The function you want to execute repeatedly."""
    metronome_interval_target = 0.526
    correction_time = 0


    prev_theta_a, prev_theta_b = angle_pairs[0]
    last_valid_measurement_results = [1,1,-1,-1]
    for i in range(cycles):
        OSC.send_click_reference(Clemens)
        theta_a, theta_b = random.choice(angle_pairs)
        #theta_a, theta_b = [Q_angles[0:2][i%2], Q_angles[2:4][i%2]]
        t = time.perf_counter() 

        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, metronome_interval=metronome_interval_target - correction_time/1.3, integration_time=0.065, max_integration_time=0.07, max_rotation_duration=0.35)

        # incase no coincidence is seen during integration time, repeat last measurement (only happens rarely)
        if measurement_results == -1:
            print("No Coincidence")
            measurement_results == last_valid_measurement_results
        else:
            last_valid_measurement_results = measurement_results

        send_results(last_valid_measurement_results, theta_a, theta_b)
        correction_time = time.perf_counter() - t - metronome_interval_target

def performance_cycle_alice(cycles):
    """Only Alice flips positions every time"""
   
    prev_theta_a, prev_theta_b = angle_pairs[0]
    for i in range(cycles):
        theta_a, theta_b = [Q_angles[0:2][i%2], 0]
        t = time.perf_counter() 
        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, metronome_interval=0.500, integration_time=0.065, max_integration_time=0.07, max_rotation_duration=0.35)
        print(time.perf_counter() - t)        

def performance_cycle_bob(cycles):
    """Only Bob flips positions every time"""
   
    prev_theta_a, prev_theta_b = angle_pairs[0]
    for i in range(cycles):
        theta_a, theta_b = [0, Q_angles[2:4][i%2]]
        t = time.perf_counter() 
        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, metronome_interval=0.5, integration_time=0.065, max_integration_time=0.07, max_rotation_duration=0.35)
        print(time.perf_counter() - t)        

In [46]:
performance_cycle_both(80000)

Click sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Enar
Click sent to Clemens
Measurement [2, 2, -1, 1] sent to Clemens
Measurement [2, 2, -1, 1] sent to Enar
Click sent to Clemens
Measurement [1, 2, -1, 1] sent to Clemens
Measurement [1, 2, -1, 1] sent to Enar
Click sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Enar
Click sent to Clemens
Measurement [2, 2, 1, -1] sent to Clemens
Measurement [2, 2, 1, -1] sent to Enar
Click sent to Clemens
Measurement [1, 2, -1, -1] sent to Clemens
Measurement [1, 2, -1, -1] sent to Enar
Click sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Enar
Click sent to Clemens
Measurement [1, 1, -1, -1] sent to Clemens
Measurement [1, 1, -1, -1] sent to Enar
Click sent to Clemens
Measurement [2, 1, -1, -1] sent to Clemens
Measurement [2, 1, -1, -1] sent to Enar
Click sent to Clemens
Measurement [1, 2, 1, -1] sent to Clemens

KeyboardInterrupt: 

In [4]:
start = time.perf_counter()
result = TTC.collect_stream_data_single_attempt(0.065, 0.065)
print(time.perf_counter() - start, result )

AttributeError: 'NoneType' object has no attribute 'getChannels'

# TODO

In [8]:
start = time.perf_counter()
KMC.rotate_simulataneously_metronome(0, 45, wait_for_completion=False,target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 0, wait_for_completion=False, target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 45, wait_for_completion=False, target_duration=0.33)
KMC.rotate_simulataneously_metronome(0, 0, wait_for_completion=False, target_duration=0.33)
duration = time.perf_counter() - start
print(duration/4)

0.3302461499988567


Check with audio again, for some reason delay comp not working on single rotator rotations yet. (cw and ccw)

implement OSC sending